In [1]:
# Chefkoch.de Website
# IMPORTS
import os
import time
import datetime as dt
from datetime import datetime
from multiprocessing import Pool

from time import sleep, time
from random import randint, choice
import requests
import urllib.request
from bs4 import BeautifulSoup
import csv

import re

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
# OS
NOW           = dt.datetime.now()
FILE_NAME     = 'chefkoch_recepies_' + 'scrapped' + '.csv'
DATASET_FOLDER = '/content/drive/My Drive/Homemade RecipeBowl/Image to recipe/input/'
IMGS_FOLDER  = '/content/drive/My Drive/Homemade RecipeBowl/Image to recipe/input/images/'
Add_IMG = 6

# Chefkoch.de Seite
CHEFKOCH_URL  = 'http://www.chefkoch.de'
START_URL     = 'http://www.chefkoch.de/rs/s'
CATEGORY      = '/Rezepte.html'

In [5]:
category_url = START_URL + '0o3' + CATEGORY

def _get_html(url):
    page = ''
    while page == '':
        try:
            page = requests.get(url, headers={'User-Agent':'Mozilla/5.0'})
        except:
            print('Connection refused')
            sleep(10)
            continue
    return page.text

html_text_total_pages = _get_html(category_url)

In [6]:
url_list = []

for i in range(0, 10560 + 1):
    url_to_scrap = START_URL + str(i * 30) + 'o3' + CATEGORY
    url_list.append(url_to_scrap)

from pprint import pprint
pprint(url_list[:30])

print("Total Pages:",len(url_list))

['http://www.chefkoch.de/rs/s0o3/Rezepte.html',
 'http://www.chefkoch.de/rs/s30o3/Rezepte.html',
 'http://www.chefkoch.de/rs/s60o3/Rezepte.html',
 'http://www.chefkoch.de/rs/s90o3/Rezepte.html',
 'http://www.chefkoch.de/rs/s120o3/Rezepte.html',
 'http://www.chefkoch.de/rs/s150o3/Rezepte.html',
 'http://www.chefkoch.de/rs/s180o3/Rezepte.html',
 'http://www.chefkoch.de/rs/s210o3/Rezepte.html',
 'http://www.chefkoch.de/rs/s240o3/Rezepte.html',
 'http://www.chefkoch.de/rs/s270o3/Rezepte.html',
 'http://www.chefkoch.de/rs/s300o3/Rezepte.html',
 'http://www.chefkoch.de/rs/s330o3/Rezepte.html',
 'http://www.chefkoch.de/rs/s360o3/Rezepte.html',
 'http://www.chefkoch.de/rs/s390o3/Rezepte.html',
 'http://www.chefkoch.de/rs/s420o3/Rezepte.html',
 'http://www.chefkoch.de/rs/s450o3/Rezepte.html',
 'http://www.chefkoch.de/rs/s480o3/Rezepte.html',
 'http://www.chefkoch.de/rs/s510o3/Rezepte.html',
 'http://www.chefkoch.de/rs/s540o3/Rezepte.html',
 'http://www.chefkoch.de/rs/s570o3/Rezepte.html',
 'htt

In [8]:
def _write_to_recipes(data):
    path = DATASET_FOLDER + FILE_NAME

    with open(path, 'a', newline='') as f:
        writer = csv.writer(f)

        writer.writerow((data['recipe_id'],
                        data['recipe_name'],
                        data['difficulty'],
                        data['preparation_time'],
                        data['link'],
                        data['has_picture'],
                        data['ingredients'],
                        data['instructions']))

def _get_front_page(html):
    soup = BeautifulSoup(html, "html.parser")
    lis = soup.find_all('article')#class_="search-list-item")

    for index, li in enumerate(lis):

        # Recipe ID
        try:
            id_r = "recipe-" +  str(li.find("a").get('data-vars-recipe-id'))
        except:
            id_r = ''

        # Image Link
        try:
            img_link = li.find('amp-img')['src']

            if img_link.find("recipe_fallback_image.jpg") == -1: # Not found
                img_name = IMGS_FOLDER + str(id_r) + '.jpg'
                urllib.request.urlretrieve(img_link, img_name)
                has_pic = 'yes'
            else:
                has_pic = 'no'
        except:
            has_pic = ''

        # link
        try:
            link = li.find('a').get('href')
        except:
            link = ''

        # Scraping Ingredients and intructions
        html = _get_html(link)
        soup = BeautifulSoup(html, 'html.parser')
        page = soup.find("main")
        ingred = page.find("article", class_="recipe-ingredients")
        ingred = page.find("table", class_="ingredients")
        ingred = page.find_all("tr")

        ingred_list = []
        for tr in ingred:
            step = tr.find_all("td")
            step_ingred = ""
            for td in step:
                span = td.find("span")
                if span != None:
                    span = span.text.strip()
                    span = " ".join(re.split("\s+", span, flags=re.UNICODE))
                    step_ingred = (step_ingred + span + " ")

            if len(step_ingred) != 0:
                ingred_list.append(step_ingred)

        recipe = page.find_all("article", class_="ds-box")
        recipe = recipe[2].find("div", class_="ds-box").text.strip()
        recipe = [recipe]

        # Recipe name
        try:
            name = li.find('div').find('h2').text.strip()#.find('font').text.strip() #, class_='search-list-item-title').text.strip()
        except:
            name = ''

        # Recipe Difficulty
        try:
            difficulty = li.find('span', class_='recipe-difficulty').text
            difficulty = difficulty[1:].strip()

            if difficulty == "pfiffig":
                difficulty = "smart"
            elif difficulty == "simpel":
                difficulty = "simple"

        except:
            difficulty = ''

        # Cooking Time
        try:
            preptime = li.find('span', class_='recipe-preptime').text
            preptime = preptime[1:].strip()
        except:
            preptime = ''


        # write dictionary
        data = {'recipe_id' : id_r,
                'recipe_name' : name,
                'difficulty' : difficulty,
                'preparation_time' : preptime,
                'has_picture' : has_pic,
                'link' : link,
                'ingredients' : ingred_list,
                'instructions' : recipe
                }

        # append file
        _write_to_recipes(data)

def scrap_main(url):
    print('Current url: ', url)
    html = _get_html(url)
    _get_front_page(html)
    #sleep(randint(1, 2))

start_time = time()

# count = 0
# for url in url_list[0:10561]:#10561]:
#     print("Page:",count)
#     scrap_main(url)
#     count += 1

with Pool(15) as p:
    p.map(scrap_main, url_list)

print("--- %s seconds ---" % (time() - start_time))

Current url:  http://www.chefkoch.de/rs/s0o3/Rezepte.html
Current url:  http://www.chefkoch.de/rs/s30o3/Rezepte.html
Current url:  http://www.chefkoch.de/rs/s360o3/Rezepte.html
Current url:  http://www.chefkoch.de/rs/s420o3/Rezepte.html
Current url:  http://www.chefkoch.de/rs/s300o3/Rezepte.html
Current url:  http://www.chefkoch.de/rs/s150o3/Rezepte.html
Current url:  http://www.chefkoch.de/rs/s60o3/Rezepte.html
Current url:  http://www.chefkoch.de/rs/s210o3/Rezepte.html
Current url:  http://www.chefkoch.de/rs/s270o3/Rezepte.html
Current url:  http://www.chefkoch.de/rs/s390o3/Rezepte.html
Current url:  http://www.chefkoch.de/rs/s120o3/Rezepte.html
Current url:  http://www.chefkoch.de/rs/s180o3/Rezepte.html
Current url:  http://www.chefkoch.de/rs/s330o3/Rezepte.html
Current url:  http://www.chefkoch.de/rs/s240o3/Rezepte.html
Current url:  http://www.chefkoch.de/rs/s90o3/Rezepte.html
Current url:  http://www.chefkoch.de/rs/s450o3/Rezepte.html
Current url:  http://www.chefkoch.de/rs/s480o

In [9]:
!ls /content/drive/My\ Drive/Homemade\ RecipeBowl/Image\ to\ recipe/input

chefkoch_recepies_scrapped.csv	csv_files  images  Scrapping_Content.ipynb


In [10]:
import pandas as pd
chef_rezepte = pd.read_csv('/content/drive/My Drive/Homemade RecipeBowl/Image to recipe/input/chefkoch_recepies_scrapped.csv', header=None, encoding="ISO-8859-1")
chef_rezepte.head()

,0,1,2,3,4,5,6,7
0,recipe-3939301603099336,krÃ¼meltigers rauchiger Spitzkohleintopf,normal,5 Min.,https://www.chefkoch.de/rezepte/39393016030993...,yes,"['1 groÃer Spitzkohl, ca. 1,5 kg ', '500 g To...",['Beim Spitzkohl evtl. schlechte BlÃ¤tter entf...
1,recipe-3904121594991313,KrÃ¤utersalz-Mischung,simple,10 Min.,https://www.chefkoch.de/rezepte/39041215949913...,yes,"['80 g Meersalz ', '2 TL Petersilie, getrockne...",['Das Salz und die GewÃ¼rze abwiegen und abmes...
2,recipe-3930821601311034,Rote Bete Ravioli,normal,25 Min.,https://www.chefkoch.de/rezepte/39308216013110...,yes,"['1 groÃe Rote Bete ', '50 g Ricotta ', '50 g...",['1. Zuerst Schalotten und Ingwer fein schneid...
3,recipe-3896951593524334,"Quiche mit CrÃ¨me fraÃ®che, Speck, Tomaten, KÃ...",simple,10 Min.,https://www.chefkoch.de/rezepte/38969515935243...,yes,"['1 Pck. Quicheteig, rechteckig ', '150 ml CrÃ...",['Den Backofen auf 200 Â°C HeiÃluft vorheizen...
4,recipe-3844961585738609,KrÃ¤uterbrot mittelalterlich,normal,25 Min.,https://www.chefkoch.de/rezepte/38449615857386...,yes,"['500 g Weizenmehl Type 405 ', '1 Pck. Trocken...",['Alle Zutaten von Weizenmehl bis italienische...


In [11]:
chef_rezepte[[4]][:10] 

,4
0,https://www.chefkoch.de/rezepte/39393016030993...
1,https://www.chefkoch.de/rezepte/39041215949913...
2,https://www.chefkoch.de/rezepte/39308216013110...
3,https://www.chefkoch.de/rezepte/38969515935243...
4,https://www.chefkoch.de/rezepte/38449615857386...
5,https://www.chefkoch.de/rezepte/38907815921498...
6,https://www.chefkoch.de/rezepte/38404115851325...
7,https://www.chefkoch.de/rezepte/39065715955061...
8,https://www.chefkoch.de/rezepte/38695615889468...
9,https://www.chefkoch.de/rezepte/38213215815315...


In [12]:
chef_rezepte.columns = ['recipe_id', 'recipe_name','difficulty', 'preparation_time', 'link', 'has_picture','ingredients','instructions']

In [13]:
chef_rezepte.head()

,recipe_id,recipe_name,difficulty,preparation_time,link,has_picture,ingredients,instructions
0,recipe-3939301603099336,krÃ¼meltigers rauchiger Spitzkohleintopf,normal,5 Min.,https://www.chefkoch.de/rezepte/39393016030993...,yes,"['1 groÃer Spitzkohl, ca. 1,5 kg ', '500 g To...",['Beim Spitzkohl evtl. schlechte BlÃ¤tter entf...
1,recipe-3904121594991313,KrÃ¤utersalz-Mischung,simple,10 Min.,https://www.chefkoch.de/rezepte/39041215949913...,yes,"['80 g Meersalz ', '2 TL Petersilie, getrockne...",['Das Salz und die GewÃ¼rze abwiegen und abmes...
2,recipe-3930821601311034,Rote Bete Ravioli,normal,25 Min.,https://www.chefkoch.de/rezepte/39308216013110...,yes,"['1 groÃe Rote Bete ', '50 g Ricotta ', '50 g...",['1. Zuerst Schalotten und Ingwer fein schneid...
3,recipe-3896951593524334,"Quiche mit CrÃ¨me fraÃ®che, Speck, Tomaten, KÃ...",simple,10 Min.,https://www.chefkoch.de/rezepte/38969515935243...,yes,"['1 Pck. Quicheteig, rechteckig ', '150 ml CrÃ...",['Den Backofen auf 200 Â°C HeiÃluft vorheizen...
4,recipe-3844961585738609,KrÃ¤uterbrot mittelalterlich,normal,25 Min.,https://www.chefkoch.de/rezepte/38449615857386...,yes,"['500 g Weizenmehl Type 405 ', '1 Pck. Trocken...",['Alle Zutaten von Weizenmehl bis italienische...


In [14]:
chef_rezepte.to_csv('/content/drive/My Drive/Homemade RecipeBowl/Image to recipe/input/chefkoch_recepies_scrapped-processed.csv')

In [ ]:
import os
path = '/content/drive/My Drive/Homemade RecipeBowl/Image to recipe/input/images/6'
files = os.listdir(path)
i = 1
for file in files:
    filename, file_extension = os.path.splitext(file)
    os.rename(os.path.join(path, file), os.path.join(path, filename + '-0' + file_extension))
    print('renamed: ', i)
    i = i+1

Streaming output truncated to the last 5000 lines.
renamed:  34208
renamed:  34209
renamed:  34210
renamed:  34211
renamed:  34212
renamed:  34213
renamed:  34214
renamed:  34215
renamed:  34216
renamed:  34217
renamed:  34218
renamed:  34219
renamed:  34220
renamed:  34221
renamed:  34222
renamed:  34223
renamed:  34224
renamed:  34225
renamed:  34226
renamed:  34227
renamed:  34228
renamed:  34229
renamed:  34230
renamed:  34231
renamed:  34232
renamed:  34233
renamed:  34234
renamed:  34235
renamed:  34236
renamed:  34237
renamed:  34238
renamed:  34239
renamed:  34240
renamed:  34241
renamed:  34242
renamed:  34243
renamed:  34244
renamed:  34245
renamed:  34246
renamed:  34247
renamed:  34248
renamed:  34249
renamed:  34250
renamed:  34251
renamed:  34252
renamed:  34253
renamed:  34254
renamed:  34255
renamed:  34256
renamed:  34257
renamed:  34258
renamed:  34259
renamed:  34260
renamed:  34261
renamed:  34262
renamed:  34263
renamed:  34264
renamed:  34265
renamed:  34266
renam